In [1]:
from gensim import downloader
from gensim.similarities.annoy import AnnoyIndexer
import os
import json
import gradio as gr

In [2]:
target_words = dict()
with open('./targets.json', 'r') as file:
    target_words = json.load(file)

In [7]:
num_trees = 10

print("Downloading word2vec model", end="\r")

model = downloader.load('word2vec-google-news-300')

print("Downloaded word2vec model ")

file_name = f'./tmp/my_model_{num_trees}'

if os.path.exists(file_name):
    print("Loading random forest for approximate nearest neighbors")
    annoy_index = AnnoyIndexer()
    annoy_index.load(file_name)
    annoy_index.model = model
else:
    print("Creating random forest for approximate nearest neighbors")   
    annoy_index = AnnoyIndexer(model, num_trees)
    # Persist index to disk
    annoy_index.save(file_name)

print("All set!")

Downloaded word2vec model 
Creating random forest for approximate nearest neighbors
All set!


In [18]:
input_values = []
nearest_neighbors = dict()
target = None

def choose_model(is_junior: bool = True) -> str:
    target_level = "junior" if is_junior else "normal"
    global target
    target = target_words[target_level]
    closest_words = model.most_similar([target], topn=100, indexer=annoy_index)
    global nearest_neighbors
    for i, (neighbor,_) in enumerate(closest_words):
        nearest_neighbors[neighbor] = i+1
    return f"Chose {target_level}"

def on_text_submit(guess: str) -> str:
    output_str = f"({len(input_values) + 1})"
    if guess == target:
        input_values.insert(0, f"{output_str} SUCCESS {target}!")
    else:
        try:
            approx_direction = model[target] - model[guess]
            approximate_neighbors = model.most_similar([approx_direction], topn=3, indexer=annoy_index)
            approximate_neighbors_str = ", ".join([x[0].lower() for x in approximate_neighbors])
            if guess in nearest_neighbors.keys():
                how_far = nearest_neighbors[guess]
                approximate_neighbors_str = f"{how_far}/100 {approximate_neighbors_str}"
            input_values.insert(0, f"{output_str} {guess}: {approximate_neighbors_str}")
        except Exception:
            input_values.insert(0, f"{output_str} {guess} not found")
    
    return "\n".join(input_values)

In [19]:
chooose_level = gr.Interface(
    fn=choose_model,
    inputs="checkbox",
    outputs="text",
)

play_game = gr.Interface(
    fn=on_text_submit,
    inputs="text",
    outputs="text",
    )

demo = gr.TabbedInterface(
    [chooose_level, play_game],
    ["Choose Level", "Play Game"],
)  
demo.launch()

/Users/nathanluskey/opt/anaconda3/envs/semantle/lib/python3.12/site-packages/gradio/blocks.py:1033: UserWarning: api_name predict already exists, using predict_1
  warnings.warn(


Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.


/Users/nathanluskey/opt/anaconda3/envs/semantle/lib/python3.12/site-packages/gradio/components/button.py:89: UserWarning: Using the update method is deprecated. Simply return a new object instead, e.g. `return gr.Button(...)` instead of `return gr.Button.update(...)`.
  warnings.warn(
